## Model Evaluation 

In [26]:
# import dependencies 
import math 

In [1]:
# Define input variables  
groups = [[1, 2, 3], [1], [0, 2]] 
votes = [5, 5, 5, 5]

In [2]:
# Calculate group membership 
def group_memberships(groups, votes):
    """
    Define group memberships for each participant
    :groups (list of lists): a list denotes the group and contains its members 
    :votes (list): number of participant's votes for a given project 
    :returns (list of lists): retunrs a list of group membersips for each participant  
    """
    group_memberships = [[] for _ in range(len(votes))]

    for i in range(len(groups)):
        for j in groups[i]:
            group_memberships[j].append(i)
        
    return group_memberships

In [3]:
print(group_memberships(groups, votes))

[[2], [0, 1], [0, 2], [0]]


In [ ]:
def common_group(i, j):
    """
    Define an identifyer indicating whether two participants share any common group. 
    param: i denotes a voter not equal to a voter j 
    :param: j denotes a voter not equal to a voter i
    :returns (bool): returns true if two participants share a common group and false otherwise.  
    """
    common_group = any(group in group_memberships[j] for group in group_memberships[i])

    return common_group

In [19]:
def cluster_match(groups, contributions):
    """
    groups: bag of groups (list of lists of integers)
    contributions: list of contributions (floats)
    group_memberships: list of sets (lists of lists) where
                        group_memberships[i] is the groups agent i belongs to
    """
    group_memberships = [[] for _ in range(len(contributions))]

    for i in range(len(groups)):
        for j in groups[i]:
            group_memberships[j].append(i)

    def common_group(i, j):
        """
        If voter i and voter j share any common group, return true. Else return false.
        """
        return any(group in group_memberships[j] for group in group_memberships[i])

    def K(i, group):
        """
        If group includes voter i, or any member of group shares a different group with voter i
        """
        if i in group or any(common_group(i, j) for j in group):
            return math.sqrt(contributions[i])
        return contributions[i]

    result = 0

    for g in groups:
        for i in g:
            result += contributions[i] / len(group_memberships[i])

    for g in groups:
        for h in groups:
            if g == h:
                continue

            term1 = sum(K(i, h) / len(group_memberships[i]) for i in g)
            term1 = math.sqrt(term1)

            term2 = sum(K(j, g) / len(group_memberships[j]) for j in h)
            term2 = math.sqrt(term2)

            result += term1 * term2

    return math.sqrt(result)

In [20]:
result = cluster_match(groups, contributions) 
print(result)

6.119546254329342
